In [14]:
import os
import numpy as np
import cv2
import json

# Set the paths to the dataset directories
data_dir = ''
cities = ['frankfurt', 'munster', 'lindau']
img_size = (256, 512)
num_classes = 19

def load_image(path):
    # Load an image from disk using OpenCV
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    # Resize the image to the desired size
    img = cv2.resize(img, img_size)
    # Convert the image from BGR to RGB format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Normalize the pixel values to the range [0, 1]
    img = img.astype(np.float32) / 255.0
    return img

def load_mask(path):
    # Load a mask from a JSON file
    with open(path, 'r') as f:
        json_data = json.load(f)
    # Create a mask image with the same dimensions as the original image
    mask = np.zeros((json_data['imgHeight'], json_data['imgWidth']), dtype=np.uint8)
    # Iterate over all objects in the annotation
    for obj in json_data['objects']:
        # Get the class label for the current object
        class_label = obj['label']
        # Get the segmentation mask for the current object
        polygon = obj['polygon']
        mask = cv2.fillPoly(mask, [polygon], class_label)
    # Resize the mask to the desired size
    mask = cv2.resize(mask, img_size, interpolation=cv2.INTER_NEAREST)
    return mask

def preprocess_city(images_dir, masks_dir):
    # Initialize empty lists to store the images and masks
    images = []
    masks = []
    # Iterate over all image files in the directory
    for img_file in os.listdir(images_dir):
        # Load the image and mask for the current file
        img_path = os.path.join(images_dir, img_file)
        mask_path = os.path.join(masks_dir, img_file.replace('leftImg8bit', 'gtFine').replace('.png', '.json'))
        img = load_image(img_path)
        mask = load_mask(mask_path)
        # Append the image and mask to the lists
        images.append(img)
        masks.append(mask)
    # Convert the lists to arrays
    images = np.array(images)
    masks = np.array(masks)
    return images, masks

def preprocess_dataset():
    # Initialize empty lists to store the images and masks for all cities
    all_images = []
    all_masks = []
    # Iterate over all cities in the dataset
    for city in cities:
        # Set the paths to the image and mask directories for the current city
        city_dir = os.path.join(data_dir, city)
        images_dir = os.path.join(city_dir, 'leftImg8bit/train')
        masks_dir = os.path.join(city_dir, 'gtFine/train')
        # Preprocess the images and masks for the current city
        images, masks = preprocess_city(images_dir, masks_dir)
        # Append the preprocessed images and masks to the lists
        all_images.append(images)
        all_masks.append(masks)
    # Concatenate the arrays for all cities along the first axis
        # Concatenate the arrays for all cities along the first axis
    all_images = np.concatenate(all_images, axis=0)
    all_masks = np.concatenate(all_masks, axis=0)
    return all_images, all_masks

# Preprocess the entire dataset for all cities
images, masks = preprocess_dataset()

# Save the preprocessed datasets to disk
np.save('images.npy', images)
np.save('masks.npy', masks)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'frankfurt\\leftImg8bit/train'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'cityscapes/train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'cityscapes/val',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

# Define the model architecture
model = keras.Sequential([
    layers.Input(shape=(256, 256, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(20, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'cityscapes/test',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

results = model.evaluate(test_generator)
print('test loss, test acc:', results)
